In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
import tensorflow as tf
from tensorflow.keras.backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
sess = tf.Session(config=config)
set_session(sess)

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:3e:00.0, compute capability: 7.0



In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import tensorflow as tf
from sklearn import metrics
from sklearn.model_selection import train_test_split
import glob
from imgaug import augmenters as iaa
import pickle

In [4]:
EPOCHS = 100
BATCH_SIZE = 16
IM_Z = 448
saved_path = "saved_models/model.h5"

In [5]:
with open('data.pickle', 'rb') as handle:
    [X_list_train, X_list_test, y_train, y_test] = pickle.load(handle)

In [6]:
X_list_train, X_list_valid, y_train, y_valid = train_test_split(X_list_train, y_train, test_size=0.2, stratify=y_train, random_state=7777777)

In [7]:
seq = iaa.Sequential([
    iaa.Fliplr(0.5),
    iaa.Flipud(0.5),
    iaa.contrast.LinearContrast((0.9, 1.1)),
    iaa.Multiply((0.9, 1.1), per_channel=0.1),
    iaa.Affine(
        scale=(0.9,1.),
    )
])

In [8]:
def decode_image(filename, label=None, image_size=(IM_Z, IM_Z)):
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, image_size)
    
    if label is None:
        return image
    else:
        return image, label

def data_augment(image, label=None):
    img_dtype = image.dtype
    img_shape = tf.shape(image)
    image = tf.numpy_function(seq.augment_image, [image], img_dtype)
    image = tf.reshape(image, shape = img_shape)
    
    if label is None:
        return image
    else:
        return image, label

In [9]:
AUTO = tf.data.experimental.AUTOTUNE

train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((X_list_train, y_train))
    .map(decode_image, num_parallel_calls=AUTO)
    .cache()
    .map(data_augment, num_parallel_calls=AUTO)
    .repeat()
    .shuffle(512)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((X_list_valid, y_valid))
    .map(decode_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

In [10]:
lr_decay = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor=1/np.sqrt(10),
    patience=5
)
ckpt = tf.keras.callbacks.ModelCheckpoint(
    filepath=saved_path, 
    monitor='val_loss', 
    save_best_only=True,
    save_weights_only=True,
    verbose=1
)
early = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=10
)

In [11]:
def custom_regularizer(weights):
    return 10000. * tf.reduce_sum(tf.subtract(tf.abs(weights), weights))

backbone = tf.keras.applications.ResNet50(
    input_shape=(IM_Z, IM_Z, 3), 
    weights='imagenet', 
    include_top=False
)
gavp = tf.keras.layers.GlobalAveragePooling2D()(backbone.output)
logit = tf.keras.layers.Dense(
    1, 
    use_bias=False, 
    kernel_initializer=tf.keras.initializers.truncated_normal(mean=0.05, stddev=0.01), 
    kernel_regularizer=custom_regularizer
)(gavp)
output_tensor = tf.nn.sigmoid(tf.add(logit, tf.constant(-5.)))

model = tf.keras.Model(inputs=backbone.input, outputs=output_tensor)


model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
#     model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [12]:
history = model.fit(
    train_dataset, 
    epochs=EPOCHS, 
    callbacks=[lr_decay, ckpt, early],
    steps_per_epoch=len(y_train) // BATCH_SIZE,
    validation_data=valid_dataset
)

Train on 57 steps, validate on 15 steps
Epoch 1/100
56/57 [============================>.] - ETA: 0s - loss: 20.8450 - acc: 0.1317
Epoch 00001: val_loss improved from inf to 27.28828, saving model to saved_models/model.h5
57/57 [==============================] - 73s 1s/step - loss: 20.5615 - acc: 0.1349 - val_loss: 27.2883 - val_acc: 0.1304
Epoch 2/100
56/57 [============================>.] - ETA: 0s - loss: 2.0555 - acc: 0.4821
Epoch 00002: val_loss improved from 27.28828 to 25.29223, saving model to saved_models/model.h5
57/57 [==============================] - 16s 286ms/step - loss: 2.0270 - acc: 0.4890 - val_loss: 25.2922 - val_acc: 0.1304
Epoch 3/100
56/57 [============================>.] - ETA: 0s - loss: 0.4998 - acc: 0.7891
Epoch 00003: val_loss improved from 25.29223 to 21.59755, saving model to saved_models/model.h5
57/57 [==============================] - 9s 156ms/step - loss: 0.5012 - acc: 0.7895 - val_loss: 21.5975 - val_acc: 0.1304
Epoch 4/100
56/57 [=====================